In [1]:
import pandas as pd
from collections import Counter

df_niu = pd.read_csv("Snow_Tagging/SnowTagging_Niu_ROUND3_tagged.csv")
df_cat = pd.read_csv("Snow_Tagging/SnowTagging_Cat_ROUND3_tagged.csv")
df_kathy = pd.read_csv("Snow_Tagging/SnowTagging_Kathy_ROUND3_tagged.csv")
assert(len(df_niu) == len(df_cat) == len(df_kathy))
df_cat

,embedUrl,videoId,title,isIncluded,diagnostic,prognostic,motivational
0,https://www.youtube.com/embed/MB3p6aS4Lvk,:MB3p6aS4Lvk,#TeamTrees 8 Deadly Tree Facts! Science Sage: ...,True,[none_apply],[knowledge],[donation;community_effect]
1,https://www.youtube.com/embed/_rQrNfRww60,:_rQrNfRww60,HallowTree | DONATE TO TEAM TREES |,True,[none_apply],[creation],[donation]
2,https://www.youtube.com/embed/wSeDuvNuuy8,:wSeDuvNuuy8,#teamtrees cake! Forest/Treescape Cake,True,[none_apply],[creation],[donation;more_trees]
3,https://www.youtube.com/embed/RMPMdDBgWiw,:RMPMdDBgWiw,World Box - Creating The Entire Earth! War Spe...,False,[],[],[]
4,https://www.youtube.com/embed/OTSMHGVXoQ0,:OTSMHGVXoQ0,Playing various games that are random for #Tea...,True,[none_apply],[shout-out],[donation;celebrity_effect]
5,https://www.youtube.com/embed/5nC5aEtUmKg,:5nC5aEtUmKg,Team Trees hit 20 mil!!!,True,[none_apply],[spread_the_word],[celebrity_effect]
6,https://www.youtube.com/embed/-FaPgDFbkQw,:-FaPgDFbkQw,Terraria Team Trees! | Pringles Episode 2,True,[none_apply],[gameplay],[none_apply]
7,https://www.youtube.com/embed/KCbPdK3Dx34,:KCbPdK3Dx34,TEAMTREES HIT 20 MILLION!!!! (+ Live chat reac...,True,[none_apply],[share_videos],[none_apply]
8,https://www.youtube.com/embed/RpSIq52aNZU,:RpSIq52aNZU,#TeamTrees 20 Million trees celebration,True,[none_apply],[spread_the_word],[none_apply]
9,https://www.youtube.com/embed/E-M0d6yutIE,:E-M0d6yutIE,#teamtrees,False,[],[],[]


In [2]:
# results = []
# match_all_count = 0
# q1_count = 0
# q2_count = 0
# q3_count = 0

# for index, row_cat in df_cat.iterrows():
# #     row_niu = df_niu.loc[df_niu["videoId"]==row_cat["videoId"]]
#     row_niu = df_niu.iloc[index]
#     row_kathy = df_kathy.iloc[index]
    
#     isIncluded = row_niu["isIncluded"] & row_cat["isIncluded"] & row_kathy["isIncluded"]
#     isNotIncluded = ~row_niu["isIncluded"] & ~row_cat["isIncluded"] & ~row_kathy["isIncluded"]
#     isIncluded = "match" if isIncluded.bool() or isNotIncluded.bool() else "split"
    
#     diagnostic_cat = set(row_cat["diagnostic"][1:-1].split(";"))
#     diagnostic_niu = set(row_niu["diagnostic"][1:-1].split(";"))
#     diagnostic_kathy = set(row_kathy["diagnostic"][1:-1].split(";"))
#     diagnostic_match = diagnostic_cat & diagnostic_niu & diagnostic_kathy
#     diagnostic_match_two = {}
#     if (len(diagnostic_match)==0):
#         diagnostic_match_two = {"CatandNiu":(diagnostic_cat & diagnostic_niu) or "no_common",
#                                 "CatandKathy":(diagnostic_cat & diagnostic_kathy) or "no_common",
#                                 "KathyandNiu":(diagnostic_niu & diagnostic_kathy) or "no_common"}
#     d_matchAll = False
#     if len(diagnostic_match) == 0:
#         diagnostic_match = {}
#     if len(diagnostic_match) == len(diagnostic_cat) == len(diagnostic_niu) == len(diagnostic_kathy):
#         d_matchAll = True
#         q1_count+=1
        
#     prognostic_cat = set(row_cat["prognostic"][1:-1].split(";"))
#     prognostic_niu = set(row_niu["prognostic"][1:-1].split(";"))
#     prognostic_kathy = set(row_kathy["prognostic"][1:-1].split(";"))
#     prognostic_match = prognostic_cat & prognostic_niu & prognostic_kathy
#     prognostic_match_two = {}
#     if (len(prognostic_match)==0):
#         prognostic_match_two = {"CatandNiu":(prognostic_cat & prognostic_niu) or "no_common",
#                                 "CatandKathy":(prognostic_cat & prognostic_kathy) or "no_common",
#                                 "KathyandNiu":(prognostic_niu & prognostic_kathy) or "no_common"}
#     p_matchAll = False
#     if len(prognostic_match) == 0:
#         prognostic_match = {}
#     if len(prognostic_match) == len(prognostic_cat) == len(prognostic_niu) == len(prognostic_kathy):
#         p_matchAll = True
#         q2_count+=1
        
#     motivational_cat = set(row_cat["motivational"][1:-1].split(";"))
#     motivational_niu = set(row_niu["motivational"][1:-1].split(";"))
#     motivational_kathy = set(row_kathy["motivational"][1:-1].split(";"))
#     motivational_match = motivational_cat & motivational_niu & motivational_kathy
#     motivational_match_two = {}
#     if (len(motivational_match)==0):
#         motivational_match_two = {"CatandNiu":(motivational_cat & motivational_niu) or "no_common",
#                                 "CatandKathy":(motivational_cat & motivational_kathy) or "no_common",
#                                 "KathyandNiu":(motivational_niu & motivational_kathy) or "no_common"}
#     m_matchAll = False
#     if len(motivational_match) == 0:
#         motivational_match = {}
#     if len(motivational_match) == len(motivational_cat) == len(motivational_niu) == len(motivational_kathy):
#         m_matchAll = True
#         q3_count+=1
        
#     match_all = d_matchAll & p_matchAll & m_matchAll
#     if match_all:
#         match_all_count+=1
        
#     result = {
#         "embedUrl":row_cat["embedUrl"],
#         "videoId":row_cat["videoId"],
#         "title":row_cat["title"],
#         "isIncluded":isIncluded,
#         "diagnostic_inCommon": diagnostic_match,
#         "diagnostic_inCommonTwo": diagnostic_match_two,
#         "diagnostic_isMatchedAll": d_matchAll,
#         "prognostic_inCommon": prognostic_match,
#         "prognostic_inCommonTwo": prognostic_match_two,
#         "prognostic_isMatchedAll": p_matchAll,
#         "motivational_inCommon": motivational_match,
#         "motivational_inCommonTwo": motivational_match_two,
#         "motivational_isMatchedAll": m_matchAll,
#         "isMatchedAllQuestions": match_all,
#     }
#     results.append(result)

# cols = results[0].keys()
# df_results = pd.DataFrame(results, columns=cols)
# df_results.to_csv("Snow_Tagging/ROUND3_agree_rate.csv", index=False)
# print(f"All questions: {match_all_count}")
# print(f"Q1: {q1_count}")
# print(f"Q2: {q2_count}")
# print(f"Q3: {q3_count}")
# print(len(results))

In [3]:
a = [7, 2, 6]
b = [1, 2, 5]
c = [3, 1, 8]

False & False & False

False

In [28]:
KEYWORD = {"match_all": "all same",
           "match_two": "two same",
           "match_none": "none same"}
QUESTIONS = {"diagnostic":"Q1",
             "prognostic":"Q2",
             "motivational":"Q3"}

def toList(row):
    taglist = row[1:-1].split(";")
    taglist = ["video_not_included" if tag=="" else tag for tag in taglist]
    return taglist

def generateResultForRow(question_list, row_cat, row_niu, row_kathy):
    result = {
        "embedUrl":row_cat["embedUrl"],
        "videoId":row_cat["videoId"],
        "title":row_cat["title"]}
        
    for question in question_list:
        question_cat = set(toList(row_cat[question]))
        question_niu = set(toList(row_niu[question]))
        question_kathy = set(toList(row_kathy[question]))
        question_match = question_cat & question_niu & question_kathy
        question_match_two = {}
        if (len(question_match)==0):
            question_match_two =    {"CatandNiu":(question_cat & question_niu) or "no_common",
                                    "CatandKathy":(question_cat & question_kathy) or "no_common",
                                    "KathyandNiu":(question_niu & question_kathy) or "no_common"}
            for i in ["CatandNiu","CatandKathy","KathyandNiu"]:
                if question_match_two[i] == "no_common":
                    del question_match_two[i]

        question_match_all = False
        if len(question_match) == 0:
            question_match = {}
        if len(question_match) == len(question_cat) == len(question_niu) == len(question_kathy):
            question_match_all = True

        sameness = ""
        if question_match_all:
            sameness = KEYWORD["match_all"]
        elif len(question_match_two)>0:
            sameness = KEYWORD["match_two"]
        else:
            sameness = KEYWORD["match_none"]

        result[f"{QUESTIONS[question]}_Cat_tag"] = ','.join(question_cat)
        result[f"{QUESTIONS[question]}_Kathy_tag"] = ','.join(question_kathy)
        result[f"{QUESTIONS[question]}_Niu_tag"] = ','.join(question_niu)
        result[f"{QUESTIONS[question]}_same"] = sameness
        result[f"{QUESTIONS[question]}_two_same"] = ', '.join('{} : {}'.format(key, ','.join(value)) for key, value in question_match_two.items()) if len(question_match_two)>0 else ""
        
    return result

results = []
for index, row_cat in df_cat.iterrows():
    row_niu = df_niu.iloc[index]
    row_kathy = df_kathy.iloc[index]
    result = generateResultForRow(QUESTIONS, row_cat, row_niu, row_kathy)
    results.append(result)
        
# # cols = results[0].keys()
df_results = pd.DataFrame(results)
df_results.to_csv(f"Snow_Tagging/ROUND3_agree_rate_TESTING.csv", index=False)

samelist = []
samelist.append([(f"{q}",item[f"{q}_same"]) for item in results for q in QUESTIONS.values()])
[sorted(Counter(q).items(), key= lambda i:i[0]) for q in samelist]


[[(('Q1', 'all same'), 34),
  (('Q1', 'two same'), 16),
  (('Q2', 'all same'), 26),
  (('Q2', 'none same'), 8),
  (('Q2', 'two same'), 16),
  (('Q3', 'all same'), 13),
  (('Q3', 'none same'), 20),
  (('Q3', 'two same'), 17)]]